- This notebook is to build different classifier for amphiphile mixture dataset

In [1]:
import sys
sys.path.append("../src/models")
sys.path.append("../src")
sys.path.append("../")

In [59]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt

In [ ]:
# Let's start from the simple situation
# Use the original 7 amphiphiles, but change the concentration (use a decrease gradient)

In [4]:
# Get concentration information
file_path='../../data/df_input_update_ori_20240812.csv'
df_input_update_ori = pd.read_csv(file_path) 
df_input = df_input_update_ori.copy()
df_input.iloc[:,0:7] = df_input.iloc[:,0:7].applymap(lambda x: np.log1p(x))
df_input=df_input.iloc[:, 0:8]
print(df_input.shape)
df_input.head()

(336, 8)


/var/folders/wn/w203521n4h18b4_rz5rd9tw80000gp/T/ipykernel_41123/1975257681.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_input.iloc[:,0:7] = df_input.iloc[:,0:7].applymap(lambda x: np.log1p(x))


,decanoic acid_Concentration (mM),decanoate_Concentration (mM),decylamine_Concentration (mM),decyl trimethylamine_Concentration (mM),decylsulfate_Concentration (mM),decanol_Concentration (mM),monocaprin_Concentration (mM),num_vesicles
0,0.615186,0.955511,1.231101,1.552868,1.677097,0.441476,0.579418,22
1,1.487270,0.644482,0.175633,1.056053,1.216395,0.207827,0.107059,9
2,1.163151,0.705570,1.294727,1.787584,1.658228,0.159138,0.202941,6
3,1.531476,1.787584,0.381855,0.071390,1.475907,0.589175,0.113329,2
4,1.170933,1.441019,1.677097,0.885832,1.011601,0.750236,0.300105,36


In [6]:
info_path = '../../data/Info.csv'
df_info = pd.read_csv(info_path)

df = df_input.copy()
df.columns = [col.replace('_Concentration (mM)', '') for col in df.columns[:-1]] + ['num_vesicles']

column_to_name = {
    'decanoic acid': 'Decanoic acid',
    'decanoate': 'Decanoate',
    'decylamine': 'Decylamine',
    'decyl trimethylamine': 'Decyltrimethyl ammonium bromid',
    'decylsulfate': 'Decyl sodium sulfate',
    'decanol': 'Decanol',
    'monocaprin': 'Glycerol monodecanoate'
}

# Get SMILES mapping from df_info
name_to_smiles = dict(zip(df_info['Name'], df_info['SMILES']))

new_data = {}

for col in df.columns[:-1]:  # Skip num_vesicles
    chem_name = column_to_name[col]
    smiles = name_to_smiles.get(chem_name, '')
    new_data[f"{col}_SMILES"] = [smiles] * len(df)
    new_data[f"{col}_Concentration"] = df[col]

new_data['num_vesicles'] = df['num_vesicles']

df_structured = pd.DataFrame(new_data)

# Rename columns to the desired format
new_column_names = [
    'smi1', 'conc1',
    'smi2', 'conc2',
    'smi3', 'conc3',
    'smi4', 'conc4',
    'smi5', 'conc5',
    'smi6', 'conc6',
    'smi7', 'conc7',
    'vesicles_formation'
]

df_structured.columns = new_column_names

print(df_structured.shape)
df_structured.head()

(336, 15)


,smi1,conc1,smi2,conc2,smi3,conc3,smi4,conc4,smi5,conc5,smi6,conc6,smi7,conc7,vesicles_formation
0,CCCCCCCCCC(=O)O,0.615186,CCCCCCCCCC(=O)[O-],0.955511,CCCCCCCCCCN,1.231101,CCCCCCCCCC[N+](C)(C)C.[Br-],1.552868,CCCCCCCCCCOS(=O)(=O)[O-].[Na+],1.677097,CCCCCCCCCCO,0.441476,CCCCCCCCCC(=O)OCC(CO)O,0.579418,22
1,CCCCCCCCCC(=O)O,1.487270,CCCCCCCCCC(=O)[O-],0.644482,CCCCCCCCCCN,0.175633,CCCCCCCCCC[N+](C)(C)C.[Br-],1.056053,CCCCCCCCCCOS(=O)(=O)[O-].[Na+],1.216395,CCCCCCCCCCO,0.207827,CCCCCCCCCC(=O)OCC(CO)O,0.107059,9
2,CCCCCCCCCC(=O)O,1.163151,CCCCCCCCCC(=O)[O-],0.705570,CCCCCCCCCCN,1.294727,CCCCCCCCCC[N+](C)(C)C.[Br-],1.787584,CCCCCCCCCCOS(=O)(=O)[O-].[Na+],1.658228,CCCCCCCCCCO,0.159138,CCCCCCCCCC(=O)OCC(CO)O,0.202941,6
3,CCCCCCCCCC(=O)O,1.531476,CCCCCCCCCC(=O)[O-],1.787584,CCCCCCCCCCN,0.381855,CCCCCCCCCC[N+](C)(C)C.[Br-],0.071390,CCCCCCCCCCOS(=O)(=O)[O-].[Na+],1.475907,CCCCCCCCCCO,0.589175,CCCCCCCCCC(=O)OCC(CO)O,0.113329,2
4,CCCCCCCCCC(=O)O,1.170933,CCCCCCCCCC(=O)[O-],1.441019,CCCCCCCCCCN,1.677097,CCCCCCCCCC[N+](C)(C)C.[Br-],0.885832,CCCCCCCCCCOS(=O)(=O)[O-].[Na+],1.011601,CCCCCCCCCCO,0.750236,CCCCCCCCCC(=O)OCC(CO)O,0.300105,36


In [33]:
# Now get the classifier based on FM

In [37]:
from utils import run_classifier_update 
from utils import binarize_last_column

In [40]:
df_train_emb_ori, x_smi_ori = get_latent_space_c(df_structured)

Random Seed: 12345
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding


/Users/haoliu/anaconda3/envs/fm4m/lib/python3.9/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)


Vocab size: 2393
[INFERENCE MODE - smi-ted-Light]


100%|██████████| 7/7 [00:00<00:00, 32.52it/s]
/Users/haoliu/Documents/GitHub/FoundationModel_on_Liposome/code/notebooks/../src/Concentration.py:30: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_train_emb = train_df.applymap(lambda x: replace_with_list(x, train_dict))


In [44]:
df_smi = pd.concat([x_smi_ori, df_structured.iloc[:,-1:]], axis=1)
df_smi.head()

,0,1,2,3,4,5,6,7,8,9,...,759,760,761,762,763,764,765,766,767,vesicles_formation
0,2.133225,-3.162935,0.789541,2.850490,3.396664,-4.821144,8.909613,0.670195,6.191030,1.689416,...,-1.362730,1.848185,2.106291,-3.763827,3.503883,-4.127566,6.484612,4.386610,-0.428853,22
1,1.466877,-2.214961,0.532058,1.976126,2.356294,-3.329292,6.124198,0.456356,4.332312,1.173332,...,-0.989388,1.304591,1.499759,-2.588207,2.457720,-2.849032,4.486023,3.019749,-0.259270,9
2,2.070615,-3.163396,0.776341,2.803669,3.371500,-4.708689,8.816236,0.659103,6.148665,1.676969,...,-1.333328,1.801240,2.095539,-3.742690,3.464534,-4.109265,6.426855,4.348366,-0.474271,6
3,1.751605,-2.508866,0.616528,2.436253,2.772158,-4.200621,7.392144,0.559018,5.247681,1.451382,...,-1.147136,1.592210,1.751878,-3.133080,3.034003,-3.411948,5.563546,3.599442,-0.132203,2
4,2.117789,-3.197669,0.769155,2.933909,3.431240,-5.015590,9.206402,0.678636,6.277449,1.818452,...,-1.279607,1.855525,2.025816,-3.943597,3.674247,-4.324269,6.849274,4.486197,-0.368310,36


In [45]:
df_smi_classify = binarize_last_column(df_smi)
df_smi_classify.head()

,0,1,2,3,4,5,6,7,8,9,...,759,760,761,762,763,764,765,766,767,vesicles_formation
0,2.133225,-3.162935,0.789541,2.850490,3.396664,-4.821144,8.909613,0.670195,6.191030,1.689416,...,-1.362730,1.848185,2.106291,-3.763827,3.503883,-4.127566,6.484612,4.386610,-0.428853,1
1,1.466877,-2.214961,0.532058,1.976126,2.356294,-3.329292,6.124198,0.456356,4.332312,1.173332,...,-0.989388,1.304591,1.499759,-2.588207,2.457720,-2.849032,4.486023,3.019749,-0.259270,1
2,2.070615,-3.163396,0.776341,2.803669,3.371500,-4.708689,8.816236,0.659103,6.148665,1.676969,...,-1.333328,1.801240,2.095539,-3.742690,3.464534,-4.109265,6.426855,4.348366,-0.474271,1
3,1.751605,-2.508866,0.616528,2.436253,2.772158,-4.200621,7.392144,0.559018,5.247681,1.451382,...,-1.147136,1.592210,1.751878,-3.133080,3.034003,-3.411948,5.563546,3.599442,-0.132203,1
4,2.117789,-3.197669,0.769155,2.933909,3.431240,-5.015590,9.206402,0.678636,6.277449,1.818452,...,-1.279607,1.855525,2.025816,-3.943597,3.674247,-4.324269,6.849274,4.486197,-0.368310,1


In [61]:
df_smi_classify.shape

(336, 769)

In [48]:
from xgboost import XGBClassifier

def get_classifier(xtrain, ytrain):

    # Train XGBoost classifier
    xgb_clf = XGBClassifier(n_estimators=5000, learning_rate=0.01, max_depth=10, use_label_encoder=False, eval_metric='logloss')
    xgb_clf.fit(xtrain, ytrain)
    
    return xgb_clf
    

model_clf = get_classifier(df_smi_classify.iloc[:,0:768], df_smi_classify.iloc[:,-1:])

In [72]:
from sklearn.ensemble import RandomForestClassifier

def get_classifier(xtrain, ytrain):

    # Train Random Forest classifier
    rf_clf = RandomForestClassifier(random_state=42) 
    rf_clf.fit(xtrain, ytrain)

    return rf_clf


model_clf_ran = get_classifier(df_smi_classify.iloc[:,0:768], df_smi_classify.iloc[:,-1:])

/Users/haoliu/anaconda3/envs/fm4m/lib/python3.9/site-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [ ]:
# Train GaussianProcessClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF  

def get_classifier(xtrain, ytrain):

    kernel = 1.0 * RBF(1.0)

    gp_clf = GaussianProcessClassifier(kernel=kernel, random_state=42)
    gp_clf.fit(xtrain, ytrain)

    return gp_clf

model_clf_gp = get_classifier(df_smi_classify.iloc[:,0:768], df_smi_classify.iloc[:,-1:])

/Users/haoliu/anaconda3/envs/fm4m/lib/python3.9/site-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
# Save models

In [ ]:
pickle.dump(model_clf, open('../../data/Intermediate/clf_model_AL_20250505.pkl', 'wb'))

In [ ]:
pickle.dump(model_clf_ran, open('../../data/Intermediate/clf_randomclassifier_AL_20250508.pkl', 'wb'))

In [ ]:
pickle.dump(model_clf_gp, open('../../data/Intermediate/clf_gpclassifier_AL_20250508.pkl', 'wb'))